## _Work in progress (functional, but messy)_

### Loading (look loading example notebook)

##### Imports

In [1]:
# Loading 
from pathlib import Path
from os import path
import sys
from os import listdir
from os.path import isfile, join

# Custom library from WIPAR
sys.path.append(str((Path.cwd().parent))) #use local version of ci_lib during development
import ci_lib
from ci_lib.utils import snakemake_tools
from ci_lib.features import Features, Means, Raws, Covariances, AutoCovariances, Moup, AutoCorrelations, Feature_Type
from ci_lib.plotting import graph_circle_plot, plot_glassbrain_bokeh, draw_neural_activity
from ci_lib import DecompData
from ci_lib.feature_selection import RFE_pipeline
from ci_lib.networks import construct_network, construct_network_from_feat, add_bokeh_attributes

# Processing
import numpy as np
import networkx as nx

# Dict Formating
import json     
import collections
import pandas as pd

# Plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
#Helper functions
def print_dict_keys(print_dict,level=0):
    if type(print_dict) is dict:
        print(f"Level {level}:",end=" ")
        print(list(print_dict.keys()))
        print_dict_keys(next(iter(print_dict.values())),level+1)
    else:
        print(f"Example shape of Value (feature): {print_dict.shape}")

#Used to get session name on last dim
def get_key_last_dim(rec_dict):
    #Look one level ahead
    arbitary_next_dict = next(iter(rec_dict.values()))

    if type(arbitary_next_dict) is dict:
        #If still dict continue recursion with next element
        return get_key_last_dim(arbitary_next_dict)
    else:
        #Else return a key from current level
        return list(rec_dict.keys())[0]  #TODO this will break if 'All' becomes first entry as it has no parcellation and is only the aggregation of the features from different sessions


## Loading
* Feature Values (exported, can be loaded using generic numpy loading function)
* Parcellations to get labels of nodes (custom class, load with ci_lib package)

In [13]:
''' 
Example of loading exports from WIPAR:
    Load dictionary that aggregates Feature Values {Parcellation x Feature x Condition x  (Combined Sessions + Indivdual Sessions)}
'''

#Manually define export file for loading (otherwise most recent export is loaded)
overwrite_file= None 

export_path = Path.cwd().parent/"results/exports/"

if overwrite_file is not None:
    #Loads manually defined export file
    loading_path = export_path/overwrite_file
else:
    #Iterates all export files to find most recent
    files = [export_path/f for f in listdir(export_path) if isfile(join(export_path, f))]
    try:
        loading_path = files[np.argmin([os.path.getmtime(f) for f in files])]

    except ValueError as err:
        if 'empty sequence' in str(err):
            print("Export folder is empty, run the feature rule first")
            loading_path = None 

#Loading
if loading_path is not None:   

    feat_dict = np.load(loading_path, allow_pickle=True)

    print(f"Loaded feature dict from: {loading_path}")
    print("With the following structure:")
    print_dict_keys(feat_dict)


Loaded feature dict from: /home/kuehn/WIPAR/calcium-imaging-analysis/results/exports/feats_hash6de2b164e9fc7f6457b59a40fb3c768c.npy
With the following structure:
Level 0: ["anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,MOs-L,SSp-bfd-L,VISrl-L,VISp-L']", 'anatomical_ROI~[]']
Level 1: ['mean']
Level 2: ['left', 'right']
Level 3: ['GN06.2021-03-26_10-53-05', 'GN06.2021-03-29_11-51-27', 'GN06.2021-03-01_13-57-39', 'GN06.2021-03-02_15-49-02', 'GN06.2021-03-03_14-02-06', 'GN06.2021-03-04_11-51-23', 'GN09.2021-03-26_10-13-36', 'GN09.2021-04-14_16-01-37', 'GN09.2021-04-15_11-12-30', 'GN09.2021-04-15_15-21-57', 'GN09.2021-04-16_15-11-45', 'GN09.2021-04-19_10-37-14', 'GN09.2021-04-20_13-07-10', 'GN09.2021-04-21_13-20-21', 'All']
Example shape of Value (feature): (142, 133, 8)


In [11]:
''' 
Example of loading custom classes from WIPAR:
    Loads the DecompData object for an arbirtary session for each parcellation
'''

#Get arbitrary session from last dim of feat_dict
example_session =   Path(get_key_last_dim(feat_dict)) 

#Get list of parcellations and features
parcellations = list(feat_dict.keys())

#?
features = list(next(iter(feat_dict.values())).keys())

#Get labels for each parcellation
labels = {}
for parcellation_name in list(feat_dict.keys()):

    #Load parcellation from arbitrary session to get labels
    res_path = Path.cwd().parent /'results'
    parcellation_path = Path(res_path/example_session/Path(parcellation_name)/"data.h5")
    parcellation = DecompData.load(parcellation_path)

    labels[parcellation_name] = parcellation.spatial_labels


print(labels)

#Ignore that hashes do not match, I'll remove that as hashes of dicts/objects are just not deterministic in python

{"anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,MOs-L,SSp-bfd-L,VISrl-L,VISp-L']": array(['VISpᴿ   ', 'VISrlᴿ  ', 'SSp-bfdᴿ', 'MOsᴿ    ', 'MOsᴸ    ',
       'SSp-bfdᴸ', 'VISrlᴸ  ', 'VISpᴸ   '], dtype='<U8'), 'anatomical_ROI~[]': array(['FRPᴿ    ', 'MOpᴿ    ', 'MOsᴿ    ', 'SSp-nᴿ  ', 'SSp-bfdᴿ',
       'SSp-llᴿ ', 'SSp-mᴿ  ', 'SSp-ulᴿ ', 'SSp-trᴿ ', 'SSp-unᴿ ',
       'SSsᴿ    ', 'AUDdᴿ   ', 'AUDpᴿ   ', 'AUDpoᴿ  ', 'AUDvᴿ   ',
       'VISalᴿ  ', 'VISamᴿ  ', 'VISlᴿ   ', 'VISpᴿ   ', 'VISplᴿ  ',
       'VISpmᴿ  ', 'VISliᴿ  ', 'VISporᴿ ', 'ACAdᴿ   ', 'PLᴿ     ',
       'RSPaglᴿ ', 'RSPdᴿ   ', 'RSPvᴿ   ', 'VISaᴿ   ', 'VISrlᴿ  ',
       'TEaᴿ    ', 'MOBᴿ    ', 'MOBᴸ    ', 'TEaᴸ    ', 'VISrlᴸ  ',
       'VISaᴸ   ', 'RSPvᴸ   ', 'RSPdᴸ   ', 'RSPaglᴸ ', 'PLᴸ     ',
       'ACAdᴸ   ', 'VISporᴸ ', 'VISliᴸ  ', 'VISpmᴸ  ', 'VISplᴸ  ',
       'VISpᴸ   ', 'VISlᴸ   ', 'VISamᴸ  ', 'VISalᴸ  ', 'AUDvᴸ   ',
       'AUDpoᴸ  ', 'AUDpᴸ   ', 'AUDdᴸ   ', 'SSsᴸ    ', 'SSp-unᴸ ',
       'SSp-trᴸ ', 'SSp-ulᴸ ', '

### Options

In [5]:
no_self_connect = False #masks diagonal with 0


## Visualization



In [6]:
#Create dataframe containing the feature average across trial and label for each condition of a given parcellation x feature combination
def feat_to_df(parcellation,feature):
   keys, means, df, number_trials = [],[],[],[]
   maxi , mini = 0, 0
  
   for key , cond in feats[parcellation][feature].items():
      keys.append(key)
      print(key)

      mean = np.mean(np.squeeze(cond['All']), axis=0)

      #Exclude self-connectness for moup for visualization
      if "moup" in feature:
         np.fill_diagonal(mean,0)

      means.append(mean)
   
      n_trials, *_ = np.squeeze(cond['All']).shape
      number_trials.append(n_trials)

      print("Trials x (Feature):", end=" ")
      print(np.squeeze(cond['All']).shape)
      
      #Append as dataframe
      df.append(pd.DataFrame(means[-1],labels[parcellation],labels[parcellation]))

      #Define min, max over all conditions for plot axis
      mini = np.min([np.min(means[-1]), mini])
      maxi = np.max([np.max(means[-1]), maxi])

   return keys, means, df, number_trials, parcellation, feature, mini, maxi


In [7]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

#Difference Measure for Plot
def diff(x,y):
    #np.std([x,y]]
    return np.abs(y -x)

def plot_matrix_diff_2D(keys, means, df, number_trials, parcellation, feature, mini, maxi,fc_or_ec):
    offset = 1 # Cause subplots start at index 1?
    col_row_n = 1+len(df)
    types = [[{"type": "table"}]] + [[{"type": "heatmap"}]]*(col_row_n**2 - 1)
    types = np.reshape(types, (col_row_n,col_row_n)).tolist()

    fig = make_subplots(rows=col_row_n, cols=col_row_n, shared_xaxes=True, shared_yaxes=True, column_titles = [""] + keys, row_titles = [""] +  keys , column_widths = [500]*(col_row_n) , row_heights = [500]*(col_row_n),
                        specs=types, horizontal_spacing =0.02, vertical_spacing=0.02)


    for i in range(offset,offset+col_row_n):
        

        for j in range(offset,offset+col_row_n):
            if i == 1 and j==1:
                fig.add_trace( go.Table(columnwidth = [120,60],header=dict(values=['Condition', '#Trials']),
                                cells=dict(values=[keys, number_trials])), row=i, col= j)
                fig.data[-1].name = f"Number of trials"
                fig.update_traces(domain=dict(x=[0, 0.12]))
            elif i == 1:
                fig.add_trace( go.Heatmap(df_to_plotly(df[j-2]), colorscale= 'viridis', zauto=False, zmin= mini, zmax= maxi), row=i, col= j)
                fig.data[-1].name = f"μ({keys[j-2]})"

                #w , h = df[i].shape
                #fig.add_hline(y=h/2-0.5, line_dash="dash", line_color="white")
                #fig.add_vline(x=w/2-0.5, line_dash="dash", line_color="white")
            elif j == 1:
                fig.add_trace( go.Heatmap(df_to_plotly(df[i-2]), colorscale= 'viridis', zauto=False, zmin= mini, zmax= maxi), row= i, col= j)
                fig.data[-1].name = f"μ({keys[i-2]})"
            else:
                std = pd.DataFrame( diff(means[i-2],means[j-2]) ,labels[parcellation],labels[parcellation])
                fig.add_trace( go.Heatmap(df_to_plotly(std), colorscale= 'viridis', zauto=False, zmin= mini, zmax= maxi), row= i, col= j)
                fig.data[-1].name = f"|μ({keys[i-2]})<br>-μ({keys[j-2]})|"

            
    fig.update_xaxes(tickmode='linear', matches='x')
    fig.update_yaxes(tickmode='linear',autorange="reversed", matches='y')

    fig.update_layout(
        hoverlabel_namelength=-1,
        width=1500,
        height=1500,
            title=dict( 
            text=f'Mean {fc_or_ec} Brain Connectivities and their Differences <br><span style="font-size: 14px;">(Given by "{feature}" for "{parcellation}")</span>',
            x=0.5,
            y=0.985,
            font=dict(
                family="Arial",
                size=22,
                color='#000000'
            )
        ))

    fig.show()
    figures_path =output_path/f"{parcellation}_{feature}"
    fig.write_html(f"{figures_path}_{configuration_name}.html")


In [17]:
for p in parcellations:
    for f in features:   
        print(feat_dict[p][f])

        if "cov" in f:
            plot_matrix_diff_2D(*feat_to_df(p,f),"Functional")
        if "moup" in f:
            plot_matrix_diff_2D(*feat_to_df(p,f),"Effective")
        



{'left': {'GN06.2021-03-26_10-53-05': array([[[-1.50380930e-02, -1.56461407e-02, -1.32708175e-02, ...,
         -1.02330236e-02, -1.04899852e-02, -8.74270034e-03],
        [-1.34234296e-02, -1.19474018e-02, -1.15830958e-02, ...,
         -4.80533593e-03, -6.39101970e-03, -7.09675052e-03],
        [-1.34317965e-02, -1.00652679e-02, -8.85781099e-03, ...,
         -2.61248871e-03, -5.00216730e-03, -7.39450594e-03],
        ...,
        [-1.31492531e-03,  1.74613678e-03, -1.32309104e-02, ...,
         -7.73287315e-03,  7.54272663e-03,  2.54899904e-03],
        [ 1.01013771e-02,  1.20112561e-02, -3.17858514e-03, ...,
          4.94988332e-03,  2.21139214e-02,  1.63844889e-02],
        [ 1.19049725e-02,  1.32427809e-02, -3.91081848e-03, ...,
          4.53147426e-03,  2.46664641e-02,  1.94828676e-02]],

       [[ 2.66690896e-02,  2.90549517e-02,  2.68103370e-03, ...,
          6.08295470e-03,  2.78803834e-02,  2.43629444e-02],
        [ 2.95808390e-02,  3.31151935e-02,  8.07539346e-03, ...,


In [9]:
n_comps = 8
edges = list(range(int(n_comps*(n_comps+1)/2))) #range(row*n_comps,(row+1)*n_comps))

resl_path = Path(path.abspath('')) /'results'
#svd_path = Path(resl_path/"GN06_2021-03-26_10-53-05/anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,VISp-L,VISrl-L,SSp-bfd-L,MOs-L']/data.h5")
svd_path = Path(resl_path/"GN06_2021-03-26_10-53-05/anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,VISp-L,VISrl-L,SSp-bfd-L,MOs-L']/data.h5")

parcellation = DecompData.load(svd_path)
print(parcellation.spatial_labels)

#feat_path= Path(resl_path/"GN06_2021-03-26_10-53-05/anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,VISp-L,VISrl-L,SSp-bfd-L,MOs-L']/All/Features/tactile/autocovariance_timelags~[1]/features.h5")
feat_path= Path(resl_path/"GN06_2021-03-26_10-53-05/anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,VISp-L,VISrl-L,SSp-bfd-L,MOs-L']/All/Features/tactile/autocovariance_timelags~[1]/features.h5")

feat = Moup.load(feat_path)
print(feat.feature.shape)

feat_val = np.mean(feat.feature[0,:,:,:],axis=0).flatten()

cutted_labels=parcellation.spatial_labels[:n_comps] if parcellation.spatial_labels is not None else None

#max=0.02
#feat_val = np.clip(feat_val,a_min=0,a_max=max)
#feat_val = feat_val / max
feat_val =  feat_val / np.max(feat_val)
np.set_printoptions(suppress = True)
print(np.asarray(feat_val))

#feat_padded = np.zeros((n_comps,n_comps))
#feat_padded[list_best_feat]=feat_val


print("feat_val")
#print(feat_padded)
#rfe_graph = construct_network(edges, n_nodes = n_comps, feat_type = Feature_Type.UNDIRECTED, edge_weight=None)# 0.01+ 99*feat_val/100)
#nx.set_edge_attributes(rfe_graph, edge_attrs, "edge_color")

print("new f")
rfe_graph_2 = construct_network_from_feat(edges, n_comps, feat_type = Feature_Type.UNDIRECTED, feat_val = visualized_data)

rfe_graph = add_bokeh_attributes(rfe_graph_2)
print(rfe_graph)
print(rfe_graph_2)
print(rfe_graph_2.nodes[0])


plot_glassbrain_bokeh(graph=rfe_graph,components_spatials=parcellation.spatials,components_labels=cutted_labels,save_path=resl_path/"glassbrain.html",small_file=True)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/kuehn/WIPAR/calcium-imaging-analysis/examples/results/GN06_2021-03-26_10-53-05/anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,VISp-L,VISrl-L,SSp-bfd-L,MOs-L']/data.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)